# Sales Dashboard - Data Analysis

## Project Overview  
This project aims to analyze sales data and build a dashboard for tracking key sales KPIs, trends, and performance.  

In [14]:
# Essential Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, Input, Output
from sqlalchemy import create_engine, text
import calendar

In [2]:
def load_and_clean_data(filepath: str) -> pd.DataFrame:
    """
    Load sales data from CSV and perform initial cleaning steps

    Args:
        filepath: Path to the CSV file

    Returns:
        Cleaned pandas DataFrame
    """
    # Load dataset
    print("Loading data from:", filepath)
    df = pd.read_csv(filepath)

    # Display basic info
    print("\nDataset Info:")
    df.info()

    print("\nFirst few rows:")
    print(df.head())

    # Standardize column names
    df.columns = df.columns.str.lower()

    # Check and report missing values
    missing = df.isnull().sum()
    if missing.any():
        print("\nMissing values found:")
        print(missing[missing > 0])

    # Remove duplicates
    initial_rows = len(df)
    df = df.drop_duplicates()
    dropped_rows = initial_rows - len(df)
    if dropped_rows:
        print(f"\nRemoved {dropped_rows} duplicate rows")

    # Generate summary statistics
    print("\nSummary Statistics:")
    print(df.describe())

    return df

# Load and clean the dataset
data = load_and_clean_data("sales_data.csv")

Loading data from: sales_data.csv

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113036 entries, 0 to 113035
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Date              113036 non-null  object
 1   Day               113036 non-null  int64 
 2   Month             113036 non-null  object
 3   Year              113036 non-null  int64 
 4   Customer_Age      113036 non-null  int64 
 5   Age_Group         113036 non-null  object
 6   Customer_Gender   113036 non-null  object
 7   Country           113036 non-null  object
 8   State             113036 non-null  object
 9   Product_Category  113036 non-null  object
 10  Sub_Category      113036 non-null  object
 11  Product           113036 non-null  object
 12  Order_Quantity    113036 non-null  int64 
 13  Unit_Cost         113036 non-null  int64 
 14  Unit_Price        113036 non-null  int64 
 15  Profit            113036 non-null  i

## Finding out following KPIs using Python's Ploty and Dash
✅ Sales Trends (daily, monthly, yearly growth) <br>
✅ Top Products by revenue & quantity <br>
✅ Customer Segmentation (age groups, location-wise sales) <br>
✅ Seasonality Analysis (find peak sales periods)

In [ ]:
# Set the style for all plots
plt.style.use('seaborn')
sns.set_palette("husl")

def plot_sales_trends(data):
    """Plot improved sales trends with moving averages and growth rates"""
    # Convert date and prepare data
    data['date'] = pd.to_datetime(data['date'])

    # Daily Sales with Moving Averages
    daily_sales = data.groupby('date')['revenue'].sum().reset_index()
    daily_sales['7d_ma'] = daily_sales['revenue'].rolling(window=7).mean()
    daily_sales['30d_ma'] = daily_sales['revenue'].rolling(window=30).mean()

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=daily_sales['date'], y=daily_sales['revenue'], name="Daily Revenue",
                  line=dict(color='blue', width=1), opacity=0.5),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=daily_sales['date'], y=daily_sales['7d_ma'], name="7-day MA",
                  line=dict(color='red', width=2)),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=daily_sales['date'], y=daily_sales['30d_ma'], name="30-day MA",
                  line=dict(color='green', width=2)),
        secondary_y=False,
    )

    # Add figure title
    fig.update_layout(
        title_text="Daily Sales Trend with Moving Averages",
        template="plotly_white",
        hovermode="x unified",
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Date")

    # Set y-axes titles
    fig.update_yaxes(title_text="Revenue ($)", secondary_y=False)

    fig.show()

    # Monthly Sales with Growth Rate
    monthly_sales = data.groupby(data['date'].dt.to_period('M'))['revenue'].sum().reset_index()
    monthly_sales['date'] = monthly_sales['date'].astype(str)
    monthly_sales['mom_growth'] = monthly_sales['revenue'].pct_change() * 100

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Bar(x=monthly_sales['date'], y=monthly_sales['revenue'], name="Monthly Revenue",
               marker_color='skyblue'),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=monthly_sales['date'], y=monthly_sales['mom_growth'], name="MoM Growth",
                  line=dict(color='red', width=2), mode='lines+markers'),
        secondary_y=True,
    )

    fig.update_layout(
        title_text="Monthly Sales Trend with Growth Rate",
        template="plotly_white",
        hovermode="x unified",
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    fig.update_xaxes(title_text="Month")
    fig.update_yaxes(title_text="Revenue ($)", secondary_y=False)
    fig.update_yaxes(title_text="Month-over-Month Growth (%)", secondary_y=True)

    fig.show()

    # Yearly Sales with Growth Rate
    yearly_sales = data.groupby(data['date'].dt.year)['revenue'].sum().reset_index()
    yearly_sales['yoy_growth'] = yearly_sales['revenue'].pct_change() * 100

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Bar(x=yearly_sales['date'], y=yearly_sales['revenue'], name="Yearly Revenue",
               marker_color='teal', marker_line_color='black', marker_line_width=1.5),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=yearly_sales['date'], y=yearly_sales['yoy_growth'], name="YoY Growth",
                  line=dict(color='red', width=2), mode='lines+markers'),
        secondary_y=True,
    )

    fig.update_layout(
        title_text="Yearly Sales Trend with Growth Rate",
        template="plotly_white",
        hovermode="x unified",
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    fig.update_xaxes(title_text="Year")
    fig.update_yaxes(title_text="Revenue ($)", secondary_y=False)
    fig.update_yaxes(title_text="Year-over-Year Growth (%)", secondary_y=True)

    fig.show()

def plot_top_products(data):
    """Plot improved top products visualization"""
    # Top 10 Products by Revenue
    top_products_revenue = data.groupby('product')['revenue'].sum().nlargest(10).reset_index()
    top_products_quantity = data.groupby('product')['order_quantity'].sum().nlargest(10).reset_index()

    # Create subplot figure
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Top 10 Products by Revenue", "Top 10 Products by Quantity"))

    # Add revenue bars
    fig.add_trace(
        go.Bar(x=top_products_revenue['revenue'], y=top_products_revenue['product'],
               orientation='h', name="Revenue", marker_color='purple'),
        row=1, col=1
    )

    # Add quantity bars
    fig.add_trace(
        go.Bar(x=top_products_quantity['order_quantity'], y=top_products_quantity['product'],
               orientation='h', name="Quantity", marker_color='skyblue'),
        row=1, col=2
    )

    fig.update_layout(
        title_text="Top Products Analysis",
        template="plotly_white",
        showlegend=False,
        height=600
    )

    fig.update_xaxes(title_text="Revenue ($)", row=1, col=1)
    fig.update_xaxes(title_text="Quantity Sold", row=1, col=2)
    fig.update_yaxes(title_text="Product", row=1, col=1)
    fig.update_yaxes(title_text="Product", row=1, col=2)

    fig.show()

def plot_customer_segmentation(data):
    """Plot improved customer segmentation visualizations"""
    # Sales by Age Groups
    age_sales = data.groupby('age_group')['revenue'].sum().reset_index()
    location_sales = data.groupby('country')['revenue'].sum().reset_index()

    # Create subplot figure
    fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]],
                       subplot_titles=("Revenue Distribution by Age Group", "Revenue Distribution by Location"))

    # Add age group pie chart
    fig.add_trace(
        go.Pie(labels=age_sales['age_group'], values=age_sales['revenue'],
               name="Age Groups", hole=0.3),
        row=1, col=1
    )

    # Add location pie chart
    fig.add_trace(
        go.Pie(labels=location_sales['country'], values=location_sales['revenue'],
               name="Locations", hole=0.3),
        row=1, col=2
    )

    fig.update_layout(
        title_text="Customer Segmentation Analysis",
        template="plotly_white",
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    fig.show()

def plot_seasonality(data):
    """Plot improved seasonality analysis"""
    # Prepare data
    data['month'] = data['date'].dt.month
    seasonality = data.groupby('month')['revenue'].agg(['mean', 'std']).reset_index()

    # Create figure
    fig = go.Figure()

    # Add mean line
    fig.add_trace(
        go.Scatter(x=seasonality['month'], y=seasonality['mean'],
                  name="Average Revenue", line=dict(color='blue', width=2),
                  mode='lines+markers')
    )

    # Add standard deviation area
    fig.add_trace(
        go.Scatter(x=seasonality['month'], y=seasonality['mean'] + seasonality['std'],
                  fill=None, mode='lines', line_color='rgba(0,100,80,0.2)',
                  name='Standard Deviation')
    )

    fig.add_trace(
        go.Scatter(x=seasonality['month'], y=seasonality['mean'] - seasonality['std'],
                  fill='tonexty', mode='lines', line_color='rgba(0,100,80,0.2)',
                  name='Standard Deviation')
    )

    # Update layout
    fig.update_layout(
        title_text="Seasonality Analysis: Monthly Sales Pattern",
        template="plotly_white",
        xaxis=dict(
            ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
            tickvals=list(range(1, 13)),
            title="Month"
        ),
        yaxis=dict(title="Revenue ($)"),
        hovermode="x unified",
        showlegend=True,
        legend=dict(





            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    fig.show()


# Generate all improved visualizations
plot_sales_trends(data)
plot_top_products(data)
plot_customer_segmentation(data)
plot_seasonality(data)


C:\Users\User1\AppData\Local\Temp\ipykernel_12152\3411934632.py:2: MatplotlibDeprecationWarning:

The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.



##  Integrate Findings into Dash

In [13]:
data

,date,day,month,year,customer_age,age_group,customer_gender,country,state,product_category,sub_category,product,order_quantity,unit_cost,unit_price,profit,cost,revenue
0,2013-11-26,26,11,2013,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
1,2015-11-26,26,11,2015,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
2,2014-03-23,23,3,2014,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,23,45,120,1366,1035,2401
3,2016-03-23,23,3,2016,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,20,45,120,1188,900,2088
4,2014-05-15,15,5,2014,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,238,180,418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113031,2016-04-12,12,4,2016,41,Adults (35-64),M,United Kingdom,England,Clothing,Vests,"Classic Vest, S",3,24,64,112,72,184
113032,2014-04-02,2,4,2014,18,Youth (<25),M,Australia,Queensland,Clothing,Vests,"Classic Vest, M",22,24,64,655,528,1183
113033,2016-04-02,2,4,2016,18,Youth (<25),M,Australia,Queensland,Clothing,Vests,"Classic Vest, M",22,24,64,655,528,1183
113034,2014-03-04,4,3,2014,37,Adults (35-64),F,France,Seine (Paris),Clothing,Vests,"Classic Vest, L",24,24,64,684,576,1260


In [22]:
# Initialize the Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)

# Create a list of months in the correct order
months_order = ['January', 'February', 'March', 'April', 'May', 'June',
                'July', 'August', 'September', 'October', 'November', 'December']

data['month_name'] = data['month'].apply(lambda x: calendar.month_name[x])

# Define the layout
app.layout = html.Div([
    # Header
    html.Div([
        html.H1("📊 Sales Analytics Dashboard",
                style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': '20px'}),
        html.P("Interactive dashboard for analyzing sales performance and trends",
               style={'textAlign': 'center', 'color': '#7f8c8d'})
    ], className='header'),

    # Filters Section
    html.Div([
        html.Div([
            html.Label("Select Date Range", style={'fontWeight': 'bold'}),
            dcc.DatePickerRange(
                id='date-range',
                start_date=data['date'].min(),
                end_date=data['date'].max(),
                display_format='YYYY-MM-DD'
            )
        ], className='filter-item'),

        html.Div([
            html.Label("Select Month", style={'fontWeight': 'bold'}),
            dcc.Dropdown(
                id='month-filter',
                options=[{'label': i, 'value': i} for i in months_order],
                value=None,
                placeholder="Select a Month",
                clearable=True
            )
        ], className='filter-item'),

        html.Div([
            html.Label("Select Year", style={'fontWeight': 'bold'}),
            dcc.Dropdown(
                id='year-filter',
                options=[{'label': str(year), 'value': year}
                         for year in sorted(data['year'].unique())],
                value=None,
                placeholder='Select Year',
                clearable=True
            )
        ], className='filter-item'),

        html.Div([
            html.Label("Select Product Category", style={'fontWeight': 'bold'}),
            dcc.Dropdown(
                id='category-filter',
                options=[{'label': cat, 'value': cat}
                         for cat in sorted(data['product_category'].unique())],
                value=None,
                placeholder='Select Category',
                clearable=True
            )
        ], className='filter-item')
    ], className='filters-container'),

    # Key Metrics Cards
    html.Div([
        html.Div([
            html.H3("Total Revenue", className='metric-title'),
            html.H2(id='total-revenue', className='metric-value')
        ], className='metric-card'),
        html.Div([
            html.H3("Total Orders", className='metric-title'),
            html.H2(id='total-orders', className='metric-value')
        ], className='metric-card'),
        html.Div([
            html.H3("Average Order Value", className='metric-title'),
            html.H2(id='avg-order-value', className='metric-value')
        ], className='metric-card'),
        html.Div([
            html.H3("Growth Rate", className='metric-title'),
            html.H2(id='growth-rate', className='metric-value')
        ], className='metric-card')
    ], className='metrics-container'),

    # Sales Trends Section
    html.Div([
        html.H2("Sales Trends Analysis", className='section-title'),
        html.Div([
            dcc.Graph(id="sales-trend-daily", className='graph-container'),
            dcc.Graph(id="sales-trend-monthly", className='graph-container'),
            dcc.Graph(id="sales-trend-yearly", className='graph-container')
        ], className='graphs-grid')
    ], className='section'),

    # Top Products Section
    html.Div([
        html.H2("Top Products Analysis", className='section-title'),
        html.Div([
            dcc.Graph(id="top-products-by-revenue", className='graph-container'),
            dcc.Graph(id="top-products-by-quantity", className='graph-container')
        ], className='graphs-grid')
    ], className='section'),

    # Customer Segmentation Section
    html.Div([
        html.H2("Customer Segmentation", className='section-title'),
        html.Div([
            dcc.Graph(id="customer-segmentation-by-age", className='graph-container'),
            dcc.Graph(id="customer-segmentation-by-location", className='graph-container')
        ], className='graphs-grid')
    ], className='section'),

    # Seasonality Analysis Section
    html.Div([
        html.H2("Seasonality Analysis", className='section-title'),
        html.Div([
            dcc.Graph(id="seasonality-analysis", className='graph-container')
        ], className='graphs-grid')
    ], className='section')
], className='dashboard-container')

# Callback for updating key metrics
@app.callback(
    [Output('total-revenue', 'children'),
     Output('total-orders', 'children'),
     Output('avg-order-value', 'children'),
     Output('growth-rate', 'children')],
    [Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('month-filter', 'value'),
     Input('year-filter', 'value'),
     Input('category-filter', 'value')]
)
def update_metrics(start_date, end_date, selected_month, selected_year, selected_category):
    filtered_data = data.copy()

    # Apply filters
    if start_date and end_date:
        filtered_data = filtered_data[
            (filtered_data['date'] >= start_date) &
            (filtered_data['date'] <= end_date)
        ]
    if selected_month:
        filtered_data = filtered_data[filtered_data['month_name'] == selected_month]
    if selected_year:
        filtered_data = filtered_data[filtered_data['year'] == selected_year]
    if selected_category:
        filtered_data = filtered_data[filtered_data['product_category'] == selected_category]

    # Calculate metrics
    total_revenue = filtered_data['revenue'].sum()
    total_orders = len(filtered_data)
    avg_order_value = total_revenue / total_orders if total_orders > 0 else 0

    # Calculate growth rate
    if selected_year:
        prev_year = selected_year - 1
        prev_year_revenue = data[data['year'] == prev_year]['revenue'].sum()
        current_year_revenue = data[data['year'] == selected_year]['revenue'].sum()
        growth_rate = ((current_year_revenue - prev_year_revenue) / prev_year_revenue * 100) if prev_year_revenue > 0 else 0
    else:
        growth_rate = 0

    return [
        f"${total_revenue:,.2f}",
        f"{total_orders:,}",
        f"${avg_order_value:,.2f}",
        f"{growth_rate:,.1f}%"
    ]

# Callback for sales trends
@app.callback(
    [Output('sales-trend-daily', 'figure'),
     Output('sales-trend-monthly', 'figure'),
     Output('sales-trend-yearly', 'figure')],
    [Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('month-filter', 'value'),
     Input('year-filter', 'value'),
     Input('category-filter', 'value')]
)
def update_sales_trends(start_date, end_date, selected_month, selected_year, selected_category):
    filtered_data = data.copy()
    # Apply filters
    if start_date and end_date:
        filtered_data = filtered_data[
            (filtered_data['date'] >= start_date) &
            (filtered_data['date'] <= end_date)
        ]
    if selected_month:
        filtered_data = filtered_data[filtered_data['month_name'] == selected_month]
    if selected_year:
        filtered_data = filtered_data[filtered_data['year'] == selected_year]
    if selected_category:
        filtered_data = filtered_data[filtered_data['product_category'] == selected_category]

    # Daily Sales with Moving Averages
    daily_sales = filtered_data.groupby('date')['revenue'].sum().reset_index()
    daily_sales['7d_ma'] = daily_sales['revenue'].rolling(window=7).mean()
    daily_sales['30d_ma'] = daily_sales['revenue'].rolling(window=30).mean()

    fig_daily = go.Figure()
    fig_daily.add_trace(go.Scatter(x=daily_sales['date'], y=daily_sales['revenue'],
                                  name='Daily Revenue', line=dict(color='blue', width=1), opacity=0.5))
    fig_daily.add_trace(go.Scatter(x=daily_sales['date'], y=daily_sales['7d_ma'],
                                  name='7-day MA', line=dict(color='red', width=2)))
    fig_daily.add_trace(go.Scatter(x=daily_sales['date'], y=daily_sales['30d_ma'],
                                  name='30-day MA', line=dict(color='green', width=2)))
    fig_daily.update_layout(title='Daily Sales Trend with Moving Averages',
                           template='plotly_white', hovermode='x unified')

    # Monthly Sales with Growth Rate
    monthly_sales = filtered_data.groupby(filtered_data['date'].dt.to_period('M'))['revenue'].sum().reset_index()
    monthly_sales['date'] = monthly_sales['date'].astype(str)
    monthly_sales['mom_growth'] = monthly_sales['revenue'].pct_change() * 100

    fig_monthly = make_subplots(specs=[[{"secondary_y": True}]])
    fig_monthly.add_trace(go.Bar(x=monthly_sales['date'], y=monthly_sales['revenue'],
                                name='Monthly Revenue', marker_color='skyblue'),
                         secondary_y=False)
    fig_monthly.add_trace(go.Scatter(x=monthly_sales['date'], y=monthly_sales['mom_growth'],
                                    name='MoM Growth', line=dict(color='red', width=2),
                                    mode='lines+markers'),
                         secondary_y=True)
    fig_monthly.update_layout(title='Monthly Sales Trend with Growth Rate',
                             template='plotly_white', hovermode='x unified')

    # Yearly Sales with Growth Rate
    yearly_sales = filtered_data.groupby(filtered_data['date'].dt.year)['revenue'].sum().reset_index()
    yearly_sales['yoy_growth'] = yearly_sales['revenue'].pct_change() * 100

    fig_yearly = make_subplots(specs=[[{"secondary_y": True}]])
    fig_yearly.add_trace(go.Bar(x=yearly_sales['date'], y=yearly_sales['revenue'],
                               name='Yearly Revenue', marker_color='teal',
                               marker_line_color='black', marker_line_width=1.5),
                        secondary_y=False)
    fig_yearly.add_trace(go.Scatter(x=yearly_sales['date'], y=yearly_sales['yoy_growth'],
                                   name='YoY Growth', line=dict(color='red', width=2),
                                   mode='lines+markers'),
                        secondary_y=True)
    fig_yearly.update_layout(title='Yearly Sales Trend with Growth Rate',
                            template='plotly_white', hovermode='x unified')

    return fig_daily, fig_monthly, fig_yearly

# Callback for top products
@app.callback(
    [Output('top-products-by-revenue', 'figure'),
     Output('top-products-by-quantity', 'figure')],
    [Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('month-filter', 'value'),
     Input('year-filter', 'value'),
     Input('category-filter', 'value')]
)
def update_top_products(start_date, end_date, selected_month, selected_year, selected_category):
    filtered_data = data.copy()

    # Apply filters
    if start_date and end_date:
        filtered_data = filtered_data[
            (filtered_data['date'] >= start_date) &
            (filtered_data['date'] <= end_date)
        ]
    if selected_month:
        filtered_data = filtered_data[filtered_data['month_name'] == selected_month]
    if selected_year:
        filtered_data = filtered_data[filtered_data['year'] == selected_year]
    if selected_category:
        filtered_data = filtered_data[filtered_data['product_category'] == selected_category]

    # Top Products by Revenue
    top_products_revenue = filtered_data.groupby('product')['revenue'].sum().nlargest(10).reset_index()
    fig_revenue = px.bar(top_products_revenue, x='revenue', y='product',
                        orientation='h', title='Top 10 Products by Revenue',
                        template='plotly_white')

    # Top Products by Quantity
    top_products_quantity = filtered_data.groupby('product')['order_quantity'].sum().nlargest(10).reset_index()
    fig_quantity = px.bar(top_products_quantity, x='order_quantity', y='product',
                         orientation='h', title='Top 10 Products by Quantity',
                         template='plotly_white')

    return fig_revenue, fig_quantity

# Callback for customer segmentation
@app.callback(
    [Output('customer-segmentation-by-age', 'figure'),
     Output('customer-segmentation-by-location', 'figure')],
    [Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('month-filter', 'value'),
     Input('year-filter', 'value'),
     Input('category-filter', 'value')]
)
def update_customer_segmentation(start_date, end_date, selected_month, selected_year, selected_category):
    filtered_data = data.copy()

    # Apply filters
    if start_date and end_date:
        filtered_data = filtered_data[
            (filtered_data['date'] >= start_date) &
            (filtered_data['date'] <= end_date)
        ]
    if selected_month:
        filtered_data = filtered_data[filtered_data['month_name'] == selected_month]
    if selected_year:
        filtered_data = filtered_data[filtered_data['year'] == selected_year]
    if selected_category:
        filtered_data = filtered_data[filtered_data['product_category'] == selected_category]

    # Age Groups
    age_sales = filtered_data.groupby('age_group')['revenue'].sum().reset_index()
    fig_age = px.pie(age_sales, names='age_group', values='revenue',
                     title='Revenue Distribution by Age Group',
                     template='plotly_white', hole=0.3)

    # Locations
    location_sales = filtered_data.groupby('country')['revenue'].sum().reset_index()
    fig_location = px.pie(location_sales, names='country', values='revenue',
                          title='Revenue Distribution by Location',
                          template='plotly_white', hole=0.3)

    return fig_age, fig_location

# Callback for seasonality analysis
@app.callback(
    Output('seasonality-analysis', 'figure'),
    [Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('month-filter', 'value'),
     Input('year-filter', 'value'),
     Input('category-filter', 'value')]
)
def update_seasonality(start_date, end_date, selected_month, selected_year, selected_category):
    filtered_data = data.copy()

    # Apply filters
    if start_date and end_date:
        filtered_data = filtered_data[
            (filtered_data['date'] >= start_date) &
            (filtered_data['date'] <= end_date)
        ]
    if selected_month:
        filtered_data = filtered_data[filtered_data['month_name'] == selected_month]
    if selected_year:
        filtered_data = filtered_data[filtered_data['year'] == selected_year]
    if selected_category:
        filtered_data = filtered_data[filtered_data['product_category'] == selected_category]

    # Prepare data
    seasonality = filtered_data.groupby('month')['revenue'].agg(['mean', 'std']).reset_index()

    fig = go.Figure()

    # Add mean line
    fig.add_trace(go.Scatter(x=seasonality['month'], y=seasonality['mean'],
                            name='Average Revenue', line=dict(color='blue', width=2),
                            mode='lines+markers'))

    # Add standard deviation area
    fig.add_trace(go.Scatter(x=seasonality['month'], y=seasonality['mean'] + seasonality['std'],
                            fill=None, mode='lines', line_color='rgba(0,100,80,0.2)',
                            name='Standard Deviation'))

    fig.add_trace(go.Scatter(x=seasonality['month'], y=seasonality['mean'] - seasonality['std'],
                            fill='tonexty', mode='lines', line_color='rgba(0,100,80,0.2)',
                            name='Standard Deviation'))

    fig.update_layout(
        title='Seasonality Analysis: Monthly Sales Pattern',
        template='plotly_white',
        xaxis=dict(
            ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
            tickvals=list(range(1, 13)),
            title='Month'
        ),
        yaxis=dict(title='Revenue ($)'),
        hovermode='x unified'
    )

    return fig

# Add custom CSS
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>Sales Analytics Dashboard</title>
        {%favicon%}
        {%css%}
        <style>
            .dashboard-container {
                padding: 20px;
                background-color: #f8f9fa;
            }
            .header {
                margin-bottom: 30px;
            }
            .filters-container {
                display: flex;
                flex-wrap: wrap;
                gap: 20px;
                margin-bottom: 30px;
                padding: 20px;
                background-color: white;
                border-radius: 10px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
            .filter-item {
                flex: 1;
                min-width: 200px;
            }
            .metrics-container {
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                gap: 20px;
                margin-bottom: 30px;
            }
            .metric-card {
                background-color: white;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                text-align: center;
            }
            .metric-title {
                color: #7f8c8d;
                font-size: 1rem;
                margin-bottom: 10px;
            }
            .metric-value {
                color: #2c3e50;
                font-size: 1.5rem;
                margin: 0;
            }
            .section {
                margin-bottom: 30px;
                background-color: white;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
            .section-title {
                color: #2c3e50;
                margin-bottom: 20px;
            }
            .graphs-grid {
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(500px, 1fr));
                gap: 20px;
            }
            .graph-container {
                background-color: white;
                padding: 10px;
                border-radius: 10px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
# Define your PostgreSQL credentials
DB_NAME = 'sales_dashboard'
DB_USER = 'rkuku'
DB_PASSWORD = 'rkuku'
DB_HOST = 'localhost'
DB_PORT = '5432'
table_name = 'sales_data'

In [ ]:
# Connect to PostgreSQL using SQLAlchemy
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [ ]:
# Define the table creation query
create_table_query = """
CREATE TABLE IF NOT EXISTS sales_data (
    date DATE,
    day INT,
    month VARCHAR(20),
    year INT,
    customer_age INT,
    age_group VARCHAR(50),
    customer_gender VARCHAR(10),
    country VARCHAR(50),
    state VARCHAR(50),
    product_category VARCHAR(50),
    sub_category VARCHAR(50),
    product VARCHAR(100),
    order_quantity INT,
    unit_cost FLOAT,
    unit_price FLOAT,
    profit FLOAT,
    cost FLOAT,
    revenue FLOAT
);
"""

In [ ]:
# Execute the query to create the table
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()

print("Table 'sales_data' created successfully.")

In [ ]:
# Store the DataFrame in PostgreSQL
data.to_sql(table_name, engine, if_exists='append', index=False)

print("Data inserted successfully!")

In [ ]:
data